# Integración Azure ML - Tableau

## **Escenario**
Tienes un modelo desplegado en Azure ML que predice si un paciente es diabético o no (`Diabetic`) basándose en los siguientes datos:
- `PatientID`
- `Pregnancies`
- `PlasmaGlucose`
- `DiastolicBloodPressure`
- `TricepsThickness`
- `SerumInsulin`
- `BMI`
- `DiabetesPedigree`
- `Age`

El objetivo es usar Tableau para:
1. Cargar los datos de los pacientes.
2. Invocar el endpoint del modelo de Azure ML para obtener las predicciones.
3. Visualizar los resultados en un dashboard de Tableau.



## **Configuración en Tableau**

### 1. Cargar los Datos en Tableau
1. Abre Tableau Desktop.
2. Ve a **Connect > Text File** y selecciona el archivo `patients_data.csv`.
3. Arrastra los datos a la hoja de trabajo para cargarlos.



### 2. Crear una Conexión REST API
Para invocar el endpoint de Azure ML desde Tableau, puedes usar el conector **Web Data Connector (WDC)** o **TabPy** (Python Integration). Aquí usaremos **TabPy** porque es más flexible para este caso.

#### Instalar y Configurar TabPy
1. Descarga e instala **TabPy** desde [aquí](https://github.com/tableau/TabPy).
2. Inicia el servidor TabPy ejecutando el siguiente comando en tu terminal:
   ```bash
   tabpy
   ```
   Por defecto, TabPy escucha en `http://localhost:9004`.

3. Verifica que TabPy está funcionando correctamente accediendo a `http://localhost:9004` en tu navegador.



### 3. Escribir un Script Python para Invocar el Endpoint
Crea un script Python que envíe los datos al endpoint de Azure ML y devuelva las predicciones.

In [ ]:
%%writefile azure_ml_predict.py

import requests
import json

def predict_diabetes(data):
    # URL del endpoint
    endpoint_url = "https://<your-endpoint-url>.azurewebsites.net/score"
    
    # Encabezados HTTP
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer <your-api-key>"
    }
    
    # Construir el cuerpo de la solicitud
    payload = {"data": data}
    
    # Realizar la solicitud HTTP
    response = requests.post(endpoint_url, headers=headers, json=payload)
    
    # Parsear la respuesta JSON
    predictions = response.json().get("predictions", [])
    return predictions

### 4. Integrar el Script con Tableau
1. En Tableau, ve a **Help > Settings and Performance > Manage External Service Connection**.
2. Configura la conexión a TabPy:
   - Servidor: `localhost`
   - Puerto: `9004`

3. Usa el script Python en Tableau:
   - Ve a **Analysis > Create Calculated Field**.
   - Crea un campo calculado llamado `Prediction` con el siguiente código:

In [ ]:
# Código del Campo Calculado:
SCRIPT_STR(
    "import azure_ml_predict; return azure_ml_predict.predict_diabetes([_arg1, _arg2, _arg3, _arg4, _arg5, _arg6, _arg7, _arg8])",
    ATTR([Pregnancies]),
    ATTR([PlasmaGlucose]),
    ATTR([DiastolicBloodPressure]),
    ATTR([TricepsThickness]),
    ATTR([SerumInsulin]),
    ATTR([BMI]),
    ATTR([DiabetesPedigree]),
    ATTR([Age])
)

Esto enviará los datos de cada fila al modelo y devolverá las predicciones.

### 5. Cargar las Predicciones en Tableau
1. Una vez que el campo calculado esté listo, arrástralo al lienzo para ver las predicciones.
2. Ahora tendrás una columna con las predicciones (`Diabetic` o `Non-Diabetic`) para cada paciente.



## **Visualización en Tableau**

### 1. Crear un Dashboard
1. Arrastra la columna `PatientID` al lienzo.
2. Arrastra la columna de predicciones (`Prediction`) al lienzo.
3. Usa gráficos de barras o circulares para visualizar la distribución de pacientes diabéticos y no diabéticos.

### 2. Filtrar por Resultados
1. Agrega un filtro para mostrar solo los pacientes diabéticos o no diabéticos.
2. Usa controles interactivos para explorar los datos según otras columnas, como `Age` o `BMI`.



## **Automatización**

Para automatizar el proceso de actualización de las predicciones, puedes configurar un flujo de trabajo en Tableau Server:

1. Publica el dashboard en Tableau Server.
2. Configura un **scheduled refresh** para actualizar los datos automáticamente.
3. Asegúrate de que el endpoint de Azure ML esté disponible y funcione correctamente.

